In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd
from pandas import DataFrame as dframe

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd
from pandas import DataFrame as dframe

In [3]:
def SRC(feat_l_tr,fea_r_tr,feat_tr,feat_l_te): #rightの時は(feat_r_tr,feat_tr,feat_r_te)
    import cvxpy as cv
    ε=0.005
    SRC_Cla = np.zeros((feat_tr.shape[1], feat_l_te.shape[1]))
    SRC_l_Cla = np.zeros((feat_l_tr.shape[0], feat_l_te.shape[1]))
    SRC_r_Cla = np.zeros((feat_l_tr.shape[0], feat_l_te.shape[1]))  #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    answer = np.zeros(feat_l_te.shape[1])
    for i in range(feat_l_te.shape[1]):
        x = cv.Variable((feat_tr.shape[1],1)) #目的関数の次元
        objective = cv.Minimize(cv.norm(x,1))
        constraints = [(cv.norm(feat_l_te[:,i,np.newaxis] - feat_tr@x,2)**2)<=ε]
        prob = cv.Problem(objective, constraints)
        result = prob.solve()
        #classVariableから値を取るためには.valueを付ける
        SRC_Cla[:,i] = np.squeeze(x.value)
        #パラメータxを各クラスのパラメータに分ける
        SRC_l_Cla[:,i] = SRC_Cla[0:feat_l_tr.shape[0],i]
        SRC_r_Cla[:,i] = SRC_Cla[feat_l_tr.shape[0]:SRC_Cla.shape[0],i]
        #分類では
        #配列xの内、右のクラスのみに関する配列を使用して作成したy(右)、左のクラスのみに関する配列を使用して作成したy(左)の
        #残差が小さい方のクラスを割り当てる
        
    #パラメータを使って擬似的に各クラスのテスト信号の特徴ベクトルを作成
    dout_feat_l_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))
    dout_feat_r_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        dout_feat_l_te[:,i] = feat_l_tr.T@SRC_l_Cla[:,i]
    for i in range(feat_r_te.shape[1]):
        dout_feat_r_te[:,i] = feat_r_tr.T@SRC_r_Cla[:,i]
    
    #作成した擬似特徴ベクトルとの残差を求める
    R_l =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #残差を格納するための配列
    R_r =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        R_l[:,i] = feat_l_te[:,i]-dout_feat_l_te[:,i]
        R_r[:,i] = feat_l_te[:,i]-dout_feat_r_te[:,i]
        if np.linalg.norm(R_l[:,i], ord=2) < np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 1
        elif np.linalg.norm(R_l[:,i], ord=2) > np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 2
    
    return answer

In [4]:
np.seterr(divide='ignore', invalid='ignore')
#浮動小数点の計算での決め事
#ignoreにすることでエラーを無視する
#divideは0で除算した時どうするか、invalidは浮動小数点演算が無効な場合の処理

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [5]:
# np.set_printoptions(precision=30, suppress=True)
#配列の表示設定

In [6]:
data = np.load('A01T.npz')
#numpy独自のバイナリファイルで保存する

In [7]:
data.files
#npzファイルに保存されているnp配列の名称を確認

['s', 'etyp', 'epos', 'edur', 'artifacts']

In [8]:
prop = np.c_[data['etyp'], data['epos'], data['edur']]
#配列の結合
#np.c_[]==np.r_['-1, 2, 0']
#2次元以上の配列で、最も最低の次元(axisの番号が一番大きい)方向で結合する
#3次元の配列で結合する際は、第3次元で結合

In [9]:
prop.shape
#要素数を表示

(603, 3)

In [10]:
pd.options.display.max_rows = None
#表示設定の配列次元の制限を無しにする

In [11]:
pd.set_option('display.max_rows', None)
#上のコードとしていることはほどんど同じ

In [12]:
dframe(prop, columns=['etype', 'epos','edur']);
#DataFrameに結合配列を格納
#propというデータの3つの列を結合させてDataFrameに格納

In [13]:
sample_data = dframe(data['s'])

In [14]:
sample_data = sample_data.iloc[:, 0:22]
#行すべてと22列を切りだしている
#EOGのデータを抜いて、脳波だけを取り出している

In [15]:
sample_data.shape

(672528, 22)

In [16]:
sample_data.head();

In [17]:
idx_left = np.argwhere(prop[:, 0]==769)
idx_right = np.argwhere(prop[:, 0]==770)
#イベントタイプが左、右のデータを取得する
#argwhereは配列から条件にあうデータを取得する

In [18]:
idx_left = idx_left.flatten()
idx_right = idx_right.flatten()
#多次元配列を1次元に変換

In [19]:
left_pos = prop[idx_left, 1]
right_pos = prop[idx_right, 1]
#対応するイベントタイプのポジションを取得

In [20]:
left_dur = prop[idx_left, 2]
right_dur = prop[idx_right, 2]
#各イベントの継続時間を取得

In [21]:
dur = 313

In [22]:
E_left = np.zeros([left_pos.shape[0], dur, sample_data.shape[1]])
E_right = np.zeros([right_pos.shape[0], dur, sample_data.shape[1]])
#各イベントのサンプルを取得するための配列形成

In [23]:
for i in range(E_left.shape[0]):
    E_left[i]=sample_data.iloc[left_pos[i]:left_pos[i]+dur, :]
for i in range(E_right.shape[0]):
    E_right[i]=sample_data.iloc[right_pos[i]:right_pos[i]+dur, :]
#対応しているデータを切りだす
#左手や右手の開始時刻から継続時間までのデータを取得する
#1回の実験で22チャンネルのデータが入っている

In [24]:
Xcol = []
for i in range(1, 23):
    if i < 11:
        Xcol.append('EEG0'+str(i))
    else:
        Xcol.append('EEG'+str(i))
#行列の名前を作成

In [25]:
dframe(E_left[0], columns=Xcol)

,EEG01,EEG02,EEG03,EEG04,EEG05,EEG06,EEG07,EEG08,EEG09,EEG010,...,EEG13,EEG14,EEG15,EEG16,EEG17,EEG18,EEG19,EEG20,EEG21,EEG22
0,1.220703,3.808594,1.757812,3.759766,1.367188,2.099609,0.244141,4.687500,5.615234,5.615234,...,-8.935547,4.101562,2.294922,3.173828,-0.683594,-4.785156,3.125000,0.488281,-1.171875,2.099609
1,5.468750,4.052734,6.152344,1.904297,5.810547,1.464844,5.468750,5.029297,9.228516,3.759766,...,-3.125000,3.906250,6.787109,1.855469,0.830078,-2.294922,4.150391,1.220703,-0.146484,2.294922
2,4.345703,5.029297,5.566406,0.781250,3.564453,0.195312,5.468750,4.589844,8.251953,2.441406,...,-0.732422,2.343750,4.931641,2.148438,2.294922,-0.488281,2.832031,1.953125,1.562500,2.148438
3,2.294922,7.470703,2.685547,0.976562,-0.683594,1.025391,7.275391,6.152344,3.125000,0.537109,...,-1.953125,3.955078,0.683594,2.636719,0.683594,-0.048828,1.806641,1.367188,1.513672,1.806641
4,0.146484,1.757812,0.000000,-5.322266,-3.027344,-4.101562,6.982422,-1.660156,-0.781250,-6.396484,...,-4.394531,-2.539062,-2.197266,-2.441406,-3.515625,-3.662109,-2.441406,-2.246094,-1.025391,-0.830078
5,5.468750,5.126953,4.101562,2.636719,3.710938,5.078125,9.375000,2.441406,3.515625,1.269531,...,4.541016,3.173828,3.125000,5.615234,6.103516,5.175781,4.882812,6.933594,8.007812,8.886719
6,6.591797,1.611328,3.906250,2.441406,5.078125,3.417969,6.250000,-1.806641,3.466797,1.953125,...,2.685547,-1.123047,3.955078,4.736328,4.345703,1.855469,3.076172,6.250000,6.689453,7.080078
7,6.298828,4.638672,5.371094,7.568359,6.689453,6.884766,3.369141,0.732422,5.371094,6.982422,...,1.416016,-0.048828,4.980469,8.789062,7.666016,3.710938,5.712891,9.179688,9.619141,10.009766
8,13.574219,8.251953,13.769531,13.232422,16.601562,14.550781,8.496094,5.859375,15.722656,14.453125,...,16.357422,7.080078,15.625000,17.480469,18.603516,15.380859,14.257812,18.017578,18.603516,18.652344
9,16.699219,14.843750,18.994141,20.605469,20.166016,20.068359,11.230469,15.039062,21.191406,22.167969,...,11.474609,16.748047,20.263672,24.267578,22.167969,18.261719,17.578125,18.896484,17.578125,15.966797


In [26]:
E_left = np.transpose(E_left, (0, 2, 1))
E_right = np.transpose(E_right, (0, 2, 1))
#行と列を変換
dframe(E_left[0])

,0,1,2,3,4,5,6,7,8,9,...,303,304,305,306,307,308,309,310,311,312
0,1.220703,5.468750,4.345703,2.294922,0.146484,5.468750,6.591797,6.298828,13.574219,16.699219,...,12.255859,12.988281,13.427734,11.474609,10.888672,12.695312,10.742188,12.255859,15.820312,10.595703
1,3.808594,4.052734,5.029297,7.470703,1.757812,5.126953,1.611328,4.638672,8.251953,14.843750,...,8.203125,8.203125,8.642578,10.253906,13.427734,11.962891,13.964844,13.183594,17.089844,11.816406
2,1.757812,6.152344,5.566406,2.685547,0.000000,4.101562,3.906250,5.371094,13.769531,18.994141,...,8.496094,9.912109,10.888672,9.912109,11.035156,13.574219,12.451172,12.988281,15.917969,10.986328
3,3.759766,1.904297,0.781250,0.976562,-5.322266,2.636719,2.441406,7.568359,13.232422,20.605469,...,11.035156,11.035156,8.398438,10.253906,11.083984,9.375000,11.328125,11.376953,16.406250,9.179688
4,1.367188,5.810547,3.564453,-0.683594,-3.027344,3.710938,5.078125,6.689453,16.601562,20.166016,...,10.205078,10.498047,9.863281,10.156250,9.716797,11.181641,10.595703,12.548828,15.673828,9.033203
5,2.099609,1.464844,0.195312,1.025391,-4.101562,5.078125,3.417969,6.884766,14.550781,20.068359,...,12.207031,8.935547,6.152344,8.740234,8.056641,6.982422,8.935547,10.253906,15.820312,6.054688
6,0.244141,5.468750,5.468750,7.275391,6.982422,9.375000,6.250000,3.369141,8.496094,11.230469,...,0.537109,3.271484,6.542969,8.056641,10.644531,13.671875,12.402344,10.742188,10.302734,5.029297
7,4.687500,5.029297,4.589844,6.152344,-1.660156,2.441406,-1.806641,0.732422,5.859375,15.039062,...,1.464844,3.417969,5.029297,9.863281,13.183594,12.500000,15.673828,13.330078,15.673828,8.740234
8,5.615234,9.228516,8.251953,3.125000,-0.781250,3.515625,3.466797,5.371094,15.722656,21.191406,...,4.052734,7.617188,9.130859,11.230469,12.255859,13.769531,14.550781,13.134766,14.404297,9.179688
9,5.615234,3.759766,2.441406,0.537109,-6.396484,1.269531,1.953125,6.982422,14.453125,22.167969,...,7.617188,7.226562,5.859375,9.863281,11.572266,10.400391,14.208984,12.841797,16.601562,8.984375


In [27]:
print('Transform left_data and right_data dimension to 72 x 22 x 313, that is trial x n_el x samples\n')
print('left_data now has dimension of: ', E_left.shape)
print('right_data now has dimension of: ', E_right.shape)

Transform left_data and right_data dimension to 72 x 22 x 313, that is trial x n_el x samples

left_data now has dimension of:  (72, 22, 313)
right_data now has dimension of:  (72, 22, 313)


In [28]:
ntrain = 54
E_left_tr = E_left[:ntrain, :, :]
E_right_tr = E_right[:ntrain, :, :]
print('Take 54 data of each left and right data as training data\n')
print('left_tr has dimension of: ', E_left_tr.shape)
print('right_tr has dimension of: ', E_right_tr.shape)
#左右各54個ずつのデータを訓練に使用

Take 54 data of each left and right data as training data

left_tr has dimension of:  (54, 22, 313)
right_tr has dimension of:  (54, 22, 313)


In [29]:
ntest = 18
E_left_te = E_left[ntrain:ntrain+ntest, :, :]
E_right_te = E_right[ntrain:ntrain+ntest, :, :]
print('Take 18 data of each left and right data as train data\n')
print('left_te has dimension of: ', E_left_te.shape)
print('right_te has dimension of: ', E_right_te.shape)
#左右各18個ずつのデータをテストに使用

Take 18 data of each left and right data as train data

left_te has dimension of:  (18, 22, 313)
right_te has dimension of:  (18, 22, 313)


In [30]:
nel = 22
#要素数の定義

In [31]:
#変更可能(変更済み)
#各試行の空間共分散を計算
print('Compute covariance of E_left_tr and E_right_tr...\n')
Cov_left_tr = np.zeros((ntrain, nel, nel))
Cov_right_tr = np.zeros((ntrain, nel, nel))
Cov_left_sum = np.zeros((ntrain,nel))
Cov_right_sum = np.zeros((ntrain,nel))
Cov_left_average = np.zeros((ntrain,nel))
Cov_right_average = np.zeros((ntrain,nel))

#その試行での各電極の平均値を求める
for i in range(ntrain):
    for j in range(dur):
        Cov_left_sum[i,:] = Cov_left_sum[i,:] + E_left_tr[i, :, j]       
for i in range(ntrain):        
    Cov_left_average[i,:] = Cov_left_sum[i,:]/313
#各平均値を引く
for i in range(ntrain):
    for j in range(dur):
        E_left_tr[i, :, j] = E_left_tr[i, :, j] - Cov_left_average[i,:]

for i in range(ntrain):
    Cov_left_tr[i, :, :] = ((E_left_tr[i, :, :]@E_left_tr[i, :, :].T)/313)

#その試行での各電極の平均値を求める
for i in range(ntrain):
    for j in range(dur):
        Cov_right_sum[i,:] = Cov_right_sum[i,:] + E_right_tr[i, :, j]
for i in range(ntrain):
    Cov_right_average[i,:] = Cov_right_sum[i,:]/313

#各平均値を引く
for i in range(ntrain):
    for j in range(dur):
        E_right_tr[i, :, j] = E_right_tr[i, :, j] - Cov_right_average[i,:]
    
for i in range(ntrain):
    Cov_right_tr[i, :, :] = ((E_right_tr[i, :, :]@E_right_tr[i, :, :].T)/313)

#.Tは転置を表す、@は行列の積算の演算子、np.traceは対角和
print('Cov_left_tr has dimension of: ', Cov_left_tr.shape)
print('Cov_right_tr has dimension of: ', Cov_right_tr.shape)

Compute covariance of E_left_tr and E_right_tr...

Cov_left_tr has dimension of:  (54, 22, 22)
Cov_right_tr has dimension of:  (54, 22, 22)


In [32]:
print('Compute avg covariance of left and right class for training...\n')
avgCov_l_tr = np.sum(Cov_left_tr, axis=0)/Cov_left_tr.shape[0]
avgCov_r_tr = np.sum(Cov_right_tr, axis=0)/Cov_right_tr.shape[0]
#np.sum(axis=0)で行方向に足し合わせる
#各試行の値を足し合わせ全試行数で割ることで平均値を求めている
print('Cov_left_tr has dimension of: ', avgCov_l_tr.shape)
print('Cov_right_tr has dimension of: ', avgCov_r_tr.shape)

Compute avg covariance of left and right class for training...

Cov_left_tr has dimension of:  (22, 22)
Cov_right_tr has dimension of:  (22, 22)


In [33]:
#空間共分散の合成
print('Calculating composite covariance...\n')
avgCov_c_tr = avgCov_l_tr + avgCov_r_tr
print('Size of avgCov_c_tr = ', avgCov_c_tr.shape)

Calculating composite covariance...

Size of avgCov_c_tr =  (22, 22)


In [34]:
dframe(avgCov_c_tr)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,238.772229,208.415407,231.519799,240.251831,232.500994,214.848158,141.153720,174.957759,202.067659,221.316945,...,147.258515,151.917068,174.100983,179.510233,178.446393,169.880131,140.145278,150.297758,141.636233,121.073469
1,208.415407,213.792396,220.910649,223.442770,207.132605,192.800246,151.871330,187.956144,201.389515,216.024788,...,130.489822,166.293837,177.447967,181.763428,173.146227,161.138518,148.864211,153.776241,141.565560,125.650249
2,231.519799,220.910649,243.800408,245.978511,233.731665,210.527668,155.693531,193.900685,221.028704,237.614555,...,145.880536,171.609832,194.164953,197.405690,191.520844,177.812147,158.450114,166.710248,154.648231,134.598454
3,240.251831,223.442770,245.978511,263.903730,246.634556,229.195847,149.210977,195.556909,222.688683,251.434116,...,154.150183,173.411868,194.924465,207.574158,201.456587,188.039414,159.739841,171.232595,160.849666,137.094214
4,232.500994,207.132605,233.731665,246.634556,245.061483,228.337147,144.462295,183.173346,214.591499,238.494352,...,163.218391,164.018058,191.053579,199.917554,200.128915,189.920190,156.475204,169.754754,161.470836,136.761107
5,214.848158,192.800246,210.527668,229.195847,228.337147,228.586479,132.094778,170.236391,192.202040,221.357053,...,165.468249,154.387276,172.732216,187.792263,190.389869,188.171183,146.498780,160.222874,155.416931,131.647003
6,141.153720,151.871330,155.693531,149.210977,144.462295,132.094778,138.941952,148.300885,152.500385,153.623723,...,102.187719,141.555672,145.055205,139.469318,133.723506,126.743999,130.339315,129.055962,117.519221,111.256886
7,174.957759,187.956144,193.900685,195.556909,183.173346,170.236391,148.300885,182.930114,191.493275,200.705845,...,122.253494,171.084365,178.796133,180.496553,169.016621,153.589304,157.013885,158.869201,144.901185,131.951467
8,202.067659,201.389515,221.028704,222.688683,214.591499,192.202040,152.500385,191.493275,217.785496,228.127097,...,140.711597,176.765979,199.809802,201.651905,191.968182,171.546259,167.910108,175.040922,160.806665,141.728404
9,221.316945,216.024788,237.614555,251.434116,238.494352,221.357053,153.623723,200.705845,228.127097,260.255600,...,156.808425,186.817548,210.114230,225.313724,216.514101,203.270789,179.429929,192.529271,180.019294,158.790096


In [35]:
λ, V = np.linalg.eig(avgCov_c_tr)
#linalg.eigは初めに固有値を含んだ配列を返し、次に固有値ベクトルを返す
#U=Vである
#対称行列の固有ベクトルは直行行列

In [36]:
#変更可能
λ_dsc = np.sort(λ)[::-1]
idx_dsc = np.argsort(λ)[::-1]
V_dsc = V[:, idx_dsc]
#[::-1]で逆順になるので降順に並べ替えられる
#argsortは降順になるようなインデントの配列を返す
#λの固有値に対応するように固有値ベクトルVを並び替える

In [37]:
λ_dsc = np.diag(λ_dsc)
#固有値を対角線に配置することで対角化を行った状態にする

In [38]:
#白色化を行う(平均を0、分散を1にする)ことで単位行列を求める
#分散共分散行列に求めた固有値ベクトルと対角化した行列をかける
print('Computing white transformation matrix....\n')
λ_dsc_sqr = np.sqrt(np.linalg.inv(λ_dsc))
#linalg.invは逆行列を返す
P = (λ_dsc_sqr)@(V_dsc.T)
#逆行列のルートに固有ベクトルの行列を乗算
print('White transfomartion has dimension of: ', P.shape)

Computing white transformation matrix....

White transfomartion has dimension of:  (22, 22)


In [39]:
#変更可能
temp = P@avgCov_c_tr@P.T
#対角化した行列が単位行列と等しいか確認
#ちゃんと単位行列になっていない
#正規直交基底になるようにシュミレットを使えば単位行列になるはず
dframe(temp)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.000000e+00,4.387308e-16,1.049761e-16,7.207685e-16,1.392646e-16,-2.450870e-15,-3.049742e-16,-2.375920e-15,4.320158e-15,4.211715e-15,...,-5.732669e-15,-3.734223e-16,5.840212e-15,1.672005e-16,4.327609e-15,-2.448317e-15,-9.651273e-16,-4.175453e-15,2.005867e-14,2.613979e-14
1,5.211796e-16,1.000000e+00,-6.782533e-16,-9.948134e-16,6.192455e-16,2.176234e-16,6.828152e-16,-1.235214e-15,-1.045823e-15,-1.671666e-15,...,7.104275e-17,-7.234898e-15,-1.616672e-15,8.628673e-16,-3.719507e-15,3.822961e-16,1.960520e-15,-4.600317e-15,6.669224e-15,7.902608e-15
2,3.513885e-16,-2.717135e-16,1.000000e+00,3.292282e-16,1.424688e-15,1.318063e-16,1.103813e-16,-6.984543e-16,7.064439e-16,8.320035e-16,...,1.606501e-15,-7.483378e-16,8.009869e-16,-4.399642e-16,-2.184886e-15,-1.460757e-16,7.338084e-16,2.111685e-15,-2.814377e-15,-1.924892e-15
3,8.417551e-16,-7.332796e-17,6.294045e-16,1.000000e+00,2.894858e-15,-2.678257e-15,-1.664452e-16,-3.390054e-15,-2.668138e-15,4.573203e-16,...,-7.204154e-15,-1.068182e-15,9.963897e-16,3.872547e-15,6.991884e-15,-5.802330e-15,4.463161e-15,1.403099e-16,6.559977e-16,2.091777e-16
4,-1.204016e-16,4.446418e-16,1.584054e-15,3.238359e-15,1.000000e+00,-3.101756e-16,-2.655818e-15,-7.418449e-16,-1.742053e-15,-6.925681e-16,...,1.690726e-15,-4.333876e-15,5.229443e-16,-5.213887e-15,4.061084e-16,1.980616e-15,-1.245889e-15,2.161673e-15,-4.064392e-15,4.843620e-16
5,-2.505224e-15,4.924247e-16,7.816616e-16,-3.184348e-15,4.803817e-16,1.000000e+00,3.735763e-15,3.261875e-15,8.903169e-15,4.798142e-16,...,1.253785e-14,3.316958e-15,-1.155531e-15,-7.348857e-15,-4.401881e-15,7.612953e-15,-2.755814e-15,1.337390e-15,1.064071e-15,-2.249652e-15
6,4.703322e-16,1.312036e-15,1.891123e-16,5.558306e-17,-3.175033e-16,3.512744e-15,1.000000e+00,-5.374148e-15,-5.843086e-15,-9.206647e-16,...,-5.167264e-15,2.036351e-15,-1.323983e-15,3.097657e-15,5.811090e-15,-4.961892e-15,-2.152300e-15,-3.341434e-15,1.943032e-15,-8.679791e-16
7,-2.636378e-15,-7.376083e-16,-3.207554e-16,-3.758182e-15,1.711653e-15,3.358657e-15,-5.906454e-15,1.000000e+00,3.691688e-14,1.115347e-14,...,1.794872e-14,4.146928e-15,-4.667951e-15,-2.433096e-15,-5.978782e-16,4.885643e-15,2.187464e-15,5.475802e-15,-4.220688e-15,-3.090296e-15
8,5.054731e-15,-1.357513e-15,5.150082e-16,-2.323180e-15,5.181112e-16,7.711550e-15,-4.486528e-15,3.620119e-14,1.000000e+00,9.527657e-14,...,-1.301746e-14,8.764185e-15,-4.971058e-15,3.151198e-14,5.143409e-15,-9.042231e-15,1.474266e-14,1.311954e-14,2.430656e-14,4.034285e-15
9,5.358215e-15,-1.397696e-15,1.282329e-15,2.070150e-16,-2.521720e-16,1.799608e-15,-1.254403e-15,1.432116e-14,9.345362e-14,1.000000e+00,...,-2.324204e-14,1.695932e-15,-3.313722e-15,1.685401e-16,-8.512063e-17,-7.165523e-15,-3.061710e-15,5.652369e-15,-6.465310e-15,1.218905e-14


In [40]:
temp = np.diag(np.linalg.eigvals(temp))
#np.linalg.eigvalsで固有値のみを返す

In [41]:
dframe(np.abs(temp))
#np.absは配列の絶対値を返す

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
#白色化を行う
# Sl = P Cl P'
S_left = P@avgCov_l_tr@P.T
# Sr = P Cr P'
S_right = P@avgCov_r_tr@P.T

In [43]:
#白色化を行った行列から左と右の固有値行列を引き出す
λl, Bl = np.linalg.eig(S_left)
λr, Br = np.linalg.eig(S_right)

In [44]:
#昇順にソートを行う
idx_asc = λl.argsort() 
idx_asc

array([ 0,  3,  4,  7,  8,  9, 10, 15, 16, 17, 20, 21, 19, 18, 14, 13, 12,
       11,  6,  5,  2,  1], dtype=int64)

In [45]:
λl_asc = λl[idx_asc]
λl_asc

array([0.04983621, 0.4099857 , 0.41958582, 0.45306702, 0.45947626,
       0.46506992, 0.47125455, 0.47798616, 0.48194587, 0.49344766,
       0.50641095, 0.50809213, 0.51273313, 0.51731175, 0.52991726,
       0.53751163, 0.54532382, 0.54864896, 0.56122604, 0.5743673 ,
       0.61755717, 0.63170903])

In [46]:
# 降順にソートを行う
idx_dsc = λr.argsort()[::-1] # Use this index to sort eigenvector largest -> smallest
idx_dsc

array([ 0,  3,  4,  7,  8,  9, 10, 15, 16, 17, 20, 21, 19, 18, 14, 13, 12,
       11,  6,  5,  2,  1], dtype=int64)

In [47]:
λr_dsc = λr[idx_dsc]
λr_dsc

array([0.95016379, 0.5900143 , 0.58041418, 0.54693298, 0.54052374,
       0.53493008, 0.52874545, 0.52201384, 0.51805413, 0.50655234,
       0.49358905, 0.49190787, 0.48726687, 0.48268825, 0.47008274,
       0.46248837, 0.45467618, 0.45135104, 0.43877396, 0.4256327 ,
       0.38244283, 0.36829097])

In [48]:
#単位行列になることを確認
λl_asc + λr_dsc

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.])

In [49]:
# 左固有ベクトルを昇順でソート
Bl = Bl[:, idx_asc]
# 右固有ベクトルを降順にソート
Br = Br[:, idx_dsc]

In [50]:
# 固有値ベクトルに固有値行列をかけることで射影行列を取得
W1 = (Br.T@P).T
W2 = (Bl.T@P).T
W = W1

In [51]:
#変更可能
# 論文の手順に従い、m = 2に設定
# 2m = 4
# したがって、行列の最初と最後の行の最初のmを取得します
#一方のクラスにおける固有値が大きい初めの𝑚行と他方のクラスにおける固有値が大きい最後の𝑚行を抜き出して新たに行列を生成
#つまり、各クラスの特徴が顕著に出ている最初のｍ行と最後のｍ行を抜き出す
# 残りの行列を削除します
#m=1に変更
m = 2
x, y = W.shape
W_new = np.delete(W, np.s_[m+1:x-m+1], 0)
#np.deleteで行列を削除、np.s[:]で場所を指定、第三要素で軸指定

In [52]:
W_new.shape

(4, 22)

In [53]:
E_left_tr.shape, E_right_tr.shape

((54, 22, 313), (54, 22, 313))

In [54]:
# E を 72 x 22 x 750 の次元に転置
#第2要素が22でなければ変更することはわかるがなぜそれが必要なのかは不明
if E_left_tr.shape[1] != 22 :
    E_left_tr = np.transpose(E_left_tr, (0, 2, 1))
if E_right_tr.shape[1] != 22 :
    E_right_tr= np.transpose(E_right_tr, (0, 2, 1))

In [55]:
E_left_tr.shape, E_right_tr.shape

((54, 22, 313), (54, 22, 313))

In [56]:
#新たにZを格納する配列を作成
Z_left_tr = np.zeros((E_left_tr.shape[0], W_new.shape[0], E_left_tr.shape[2]))
Z_right_tr = np.zeros((E_right_tr.shape[0], W_new.shape[0], E_right_tr.shape[2]))

In [57]:
Z_left_tr.shape, Z_right_tr.shape

((54, 4, 313), (54, 4, 313))

In [58]:
#左のクラス
#射影行列を各試行の脳波に適用する
#得られる行列Zは試行数(今回は54)×2m×試行時間(今回は313)の次元となる
for i in range(Z_left_tr.shape[0]):
    Z_left_tr[i] = W_new@E_left_tr[i]

#右のクラス
for i in range(Z_right_tr.shape[0]):
    Z_right_tr[i] = W_new@E_right_tr[i]

In [59]:
#特徴ベクトルを格納するための配列を作成
feat_l_tr =  np.zeros((Z_left_tr.shape[0], Z_left_tr.shape[1], 1))
feat_r_tr =  np.zeros((Z_right_tr.shape[0], Z_right_tr.shape[1], 1))

In [60]:
#式の部分は変更可能
#特徴ベクトルを求める
#左のクラス
for i in range(Z_left_tr.shape[0]):
    var = np.var(Z_left_tr[i], ddof=1, axis=1)[:, np.newaxis]
    varsum = np.sum(var)
    feat_l_tr[i] = np.log(var/varsum);
#np.varで分散値を計算
#axisでどの軸に沿って分散を計算するか求める
#ddofで平均との偏差の2乗の和を(N-ddof)で割り、初期値ではデータ数であるNで割る。ddof=1では不偏分散を求める(つまりN-1)。
#np.newaxisによって新たにサイズが1の次元を追加する。


#右のクラス
for i in range(Z_right_tr.shape[0]):
    var = np.var(Z_right_tr[i], ddof=1, axis=1)[:, np.newaxis]
    varsum = np.sum(var)
    feat_r_tr[i] = np.log(var/varsum);

In [61]:
feat_l_tr.shape, feat_r_tr.shape

((54, 4, 1), (54, 4, 1))

In [83]:
#変更可能(変更済み)
#ここ以降はテストデータ
#各試行の空間共分散を計算
print('Compute covariance of E_left_te and E_right_te...\n')
Cov_left_te = np.zeros((ntest, nel, nel))
Cov_right_te = np.zeros((ntest, nel, nel))
Cov_left_te_sum = np.zeros((ntest,nel))
Cov_right_te_sum = np.zeros((ntest,nel))
Cov_left_te_average = np.zeros((ntest,nel))
Cov_right_te_average = np.zeros((ntest,nel))

#その試行での各電極の平均値を求める
for i in range(ntest):
    for j in range(dur):
        Cov_left_te_sum[i,:] = Cov_left_te_sum[i,:] + E_left_te[i, :, j]       
for i in range(ntest):        
    Cov_left_te_average[i,:] = Cov_left_te_sum[i,:]/313
#各平均値を引く
for i in range(ntest):
    for j in range(dur):
        E_left_te[i, :, j] = E_left_te[i, :, j] - Cov_left_te_average[i,:]

for i in range(ntest):
    Cov_left_te[i, :, :] = ((E_left_te[i, :, :]@E_left_te[i, :, :].T)/313)

#その試行での各電極の平均値を求める
for i in range(ntest):
    for j in range(dur):
        Cov_right_te_sum[i] = Cov_right_te_sum[i] + E_right_te[i, :, j]
for i in range(ntest):
    Cov_right_te_average[i,:] = Cov_right_te_sum[i,:]/313

#各平均値を引く
for i in range(ntest):
    for j in range(dur):
        E_right_te[i, :, j] = E_right_te[i, :, j] - Cov_right_te_average[i,:]
    
for i in range(ntest):
    Cov_right_te[i, :, :] = ((E_right_te[i, :, :]@E_right_te[i, :, :].T)/313)

#.Tは転置を表す、@は行列の積算の演算子、np.traceは対角和
print('Cov_left_te has dimension of: ', Cov_left_te.shape)
print('Cov_right_te has dimension of: ', Cov_right_te.shape)

Compute covariance of E_left_te and E_right_te...

Cov_left_te has dimension of:  (18, 22, 22)
Cov_right_te has dimension of:  (18, 22, 22)


In [84]:
print('Compute avg covariance of left and right class for training...\n')
avgCov_l_te = np.sum(Cov_left_te, axis=0)/Cov_left_te.shape[0]
avgCov_r_te = np.sum(Cov_right_te, axis=0)/Cov_right_te.shape[0]
#np.sum(axis=0)で行方向に足し合わせる
print('Cov_left_te has dimension of: ', avgCov_l_te.shape)
print('Cov_right_te has dimension of: ', avgCov_r_te.shape)

Compute avg covariance of left and right class for training...

Cov_left_te has dimension of:  (22, 22)
Cov_right_te has dimension of:  (22, 22)


In [85]:
#空間共分散の合成
print('Calculating composite covariance...\n')
avgCov_c_te = avgCov_l_te + avgCov_r_te
print('Size of avgCov_c_te = ', avgCov_c_te.shape)

Calculating composite covariance...

Size of avgCov_c_te =  (22, 22)


In [86]:
#テストデータの処理中に、射影行列Wは計算せず、トレーニングデータのWを使用して特徴ベクトルを計算する。
#トレーニングデータの射影行列を使用
W.shape

(22, 22)

In [87]:
# 論文の手順に従い、m = 2に設定
# 2m = 4
# したがって、行列の最初と最後の行の最初のmを取得します
# 残りの行列を削除します
#m=1に変更
m = 2
x, y = W.shape
W_new = np.delete(W, np.s_[m+1:x-m+1], 0)

In [88]:
W_new.shape

(4, 22)

In [89]:
E_left_te.shape, E_right_te.shape

((18, 22, 313), (18, 22, 313))

In [90]:
#Eを18 x 22 x 313に変更
if E_left_te.shape[1] != 22 :
    E_left_te = np.transpose(E_left_te, (0, 2, 1))
if E_right_te.shape[1] != 22 :
    E_right_te = np.transpose(E_right_te, (0, 2, 1))

In [91]:
E_left_te.shape, E_right_te.shape

((18, 22, 313), (18, 22, 313))

In [92]:
#新たにZを格納する配列を作成
Z_left_te = np.zeros((E_left_te.shape[0], W_new.shape[0], E_left_te.shape[2]))
Z_right_te = np.zeros((E_right_te.shape[0], W_new.shape[0], E_right_te.shape[2]))

In [93]:
Z_left_te.shape, Z_right_te.shape

((18, 4, 313), (18, 4, 313))

In [94]:
#左のクラス
#射影行列を各試行の脳波に適用する
#得られる行列Zは試行数(今回は18)×2m×試行時間(今回は313)の次元となる
for i in range(Z_left_te.shape[0]):
    Z_left_te[i] = W_new@E_left_te[i]

#右のクラス
for i in range(Z_right_te.shape[0]):
    Z_right_te[i] = W_new@E_right_te[i]

In [95]:
#特徴ベクトルを格納するための配列を作成
feat_l_te =  np.zeros((Z_left_te.shape[0], Z_left_te.shape[1], 1))
feat_r_te =  np.zeros((Z_right_te.shape[0], Z_right_te.shape[1], 1))

In [96]:
feat_l_te.shape

(18, 4, 1)

In [97]:
#対数を取ってないので不完全
#forのループ部分はおそらく行の4でするべき
#式の部分は変更可能
#特徴ベクトルを求める
#左のクラス
for i in range(Z_left_te.shape[0]):
    var = np.var(Z_left_te[i], ddof=1, axis=1)[:, np.newaxis]
    varsum = np.sum(var)
    feat_l_te[i] = np.log(var/varsum);
#np.varで分散値を計算
#axisでどの軸に沿って分散を計算するか求める
#ddofで平均との偏差の2乗の和を(N-ddof)で割り、初期値ではデータ数であるNで割る。ddof=1では不偏分散を求める(つまりN-1)。
#np.newaxisによって新たにサイズが1の次元を追加する。

#右のクラス
for i in range(Z_right_te.shape[0]):
    var = np.var(Z_right_te[i], ddof=1, axis=1)[:, np.newaxis]
    varsum = np.sum(var)
    feat_r_te[i] = np.log(var/varsum);
    

In [98]:
#CSPFilterに問題点あり
#SparseRepresentationClassificationの実装

In [99]:
#まず、特徴ベクトルを2次元に変換
feat_l_tr = np.squeeze(feat_l_tr);
feat_r_tr = np.squeeze(feat_r_tr);
feat_tr = np.concatenate([feat_l_tr, feat_r_tr],0)
feat_tr = feat_tr.T

In [100]:
#まず、特徴ベクトルを2次元に変換
feat_l_te = np.squeeze(feat_l_te);
feat_r_te = np.squeeze(feat_r_te);

In [101]:
feat_l_te = feat_l_te.T
feat_r_te = feat_r_te.T

In [102]:
def SRC(feat_l_tr,fea_r_tr,feat_tr,feat_l_te): #rightの時は(feat_r_tr,feat_tr,feat_r_te)
    import cvxpy as cv
    ε=0.005
    SRC_Cla = np.zeros((feat_tr.shape[1], feat_l_te.shape[1]))
    SRC_l_Cla = np.zeros((feat_l_tr.shape[0], feat_l_te.shape[1]))
    SRC_r_Cla = np.zeros((feat_l_tr.shape[0], feat_l_te.shape[1]))  #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    answer = np.zeros(feat_l_te.shape[1])
    for i in range(feat_l_te.shape[1]):
        x = cv.Variable((feat_tr.shape[1],1)) #目的関数の次元
        objective = cv.Minimize(cv.norm(x,1))
        constraints = [(cv.norm(feat_l_te[:,i,np.newaxis] - feat_tr@x,2)**2)<=ε]
        prob = cv.Problem(objective, constraints)
        result = prob.solve()
        #classVariableから値を取るためには.valueを付ける
        SRC_Cla[:,i] = np.squeeze(x.value)
        #パラメータxを各クラスのパラメータに分ける
        SRC_l_Cla[:,i] = SRC_Cla[0:feat_l_tr.shape[0],i]
        SRC_r_Cla[:,i] = SRC_Cla[feat_l_tr.shape[0]:SRC_Cla.shape[0],i]
        #分類では
        #配列xの内、右のクラスのみに関する配列を使用して作成したy(右)、左のクラスのみに関する配列を使用して作成したy(左)の
        #残差が小さい方のクラスを割り当てる
        
    #パラメータを使って擬似的に各クラスのテスト信号の特徴ベクトルを作成
    dout_feat_l_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))
    dout_feat_r_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        dout_feat_l_te[:,i] = feat_l_tr.T@SRC_l_Cla[:,i]
    for i in range(feat_r_te.shape[1]):
        dout_feat_r_te[:,i] = feat_r_tr.T@SRC_r_Cla[:,i]
    
    #作成した擬似特徴ベクトルとの残差を求める
    R_l =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #残差を格納するための配列
    R_r =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #(feat_r_te.shape[0], feat_r_te.shape[1])でもいい
    for i in range(feat_l_te.shape[1]):
        R_l[:,i] = feat_l_te[:,i]-dout_feat_l_te[:,i]
        R_r[:,i] = feat_l_te[:,i]-dout_feat_r_te[:,i]
        if np.linalg.norm(R_l[:,i], ord=2) < np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 1
        elif np.linalg.norm(R_l[:,i], ord=2) > np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 2
    
    return answer

In [105]:
answer = SRC(feat_l_tr,feat_r_tr,feat_tr,feat_l_te)
x = 0
for i in range(answer.shape[0]):
    if answer[i] == 1:
        x = x + 1
print((x/answer.shape[0])*100)

44.44444444444444
